In [1]:
using Revise

using ReachabilityAnalysis, SparseArrays, JLD2

LazySets.set_ztol(Float64, 1e-14); # remove if this is not needed for the plots

In [2]:
# LOAD model in RA#master (8/04/2021)
examples_dir = "/home/mforets/.julia/dev/ReachabilityAnalysis/examples/"
building_path = joinpath(examples_dir, "Building", "building.jld2");

In [3]:
using JLD2, FileIO

H = load(building_path)["H"];

┌ Warning: saved type OrderedCollections.OrderedDict{Symbol, Dict{String, Any}} is missing field maxprobe in workspace type; reconstructing
└ @ JLD2 /home/mforets/.julia/packages/JLD2/qncOK/src/data/reconstructing_datatypes.jl:152


In [4]:
H.ext[:variables].keys; # x1 a x48, t, u1

H.modes[1]

ConstrainedLinearControlContinuousSystem{Float64, Matrix{Float64}, Matrix{Float64}, Vector{LazySet{Float64}}, Vector{CartesianProduct{Float64, Singleton{Float64, Vector{Float64}}, HalfSpace{Float64, Vector{Float64}}}}}([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 12.708461033093819 4.866919335674403 … -5.188448853349926 0.0; 0.0 0.0 … 0.0 0.0], [0.0 0.0; 0.0 0.0; … ; 0.0 0.0; 0.0 1.0], LazySet{Float64}[], CartesianProduct{Float64, Singleton{Float64, Vector{Float64}}, HalfSpace{Float64, Vector{Float64}}}[CartesianProduct{Float64, Singleton{Float64, Vector{Float64}}, HalfSpace{Float64, Vector{Float64}}}(Singleton{Float64, Vector{Float64}}([1.0]), HalfSpace{Float64, Vector{Float64}}([-1.0], -0.8)), CartesianProduct{Float64, Singleton{Float64, Vector{Float64}}, HalfSpace{Float64, Vector{Float64}}}(Singleton{Float64, Vector{Float64}}([1.0]), HalfSpace{Float64, Vector{Float64}}([1.0], 1.0))])

In [5]:
?ConstrainedLinearControlContinuousSystem

search: ConstrainedLinearControlContinuousSystem



```
ConstrainedLinearControlContinuousSystem
```

Continuous-time linear control system with domain constraints of the form:

$$
    x(t)' = A x(t) + B u(t), \; x(t) ∈ \mathcal{X}, \; u(t) ∈ \mathcal{U} \; \forall t.
$$

### Fields

  * `A` – state matrix
  * `B` – input matrix
  * `X` – state constraints
  * `U` – input constraints


In [6]:
m = H.modes[1];

In [7]:
findall(!iszero, input_matrix(m))

2-element Vector{CartesianIndex{2}}:
 CartesianIndex(25, 1)
 CartesianIndex(49, 2)

In [8]:
input_matrix(m)[25, 1]

0.013696753869332967

In [9]:
input_matrix(m)[49, 2]

1.0

In [10]:
vars = H.ext[:variables].keys

n = 48
A = state_matrix(mode(H, 1));
A = A[1:n, 1:n] # OK

B = input_matrix(mode(H, 1))[1:n, 1];

B[25]

C = zeros(48)
C[25] = 1.0

@save "building_new.jld2" A B C

In [13]:
ff = @load "building_new.jld2" A B C

1.0

In [ ]:
function building_BLDF01()
    @load building_path A B
    n = size(A, 1)
    U = Hyperrectangle(low=[0.8], high=[1.0])
    S = @system(x' = Ax + Bu, u ∈ U, x ∈ Universe(n))

    #initial states
    center_X0 = [fill(0.000225, 10); fill(0.0, 38)]
    radius_X0 = [fill(0.000025, 10); fill(0.0, 14); 0.0001; fill(0.0, 23)]
    X0 = Hyperrectangle(center_X0, radius_X0)

    prob_BLDF01 = InitialValueProblem(S, X0)
end

In [ ]:
[fill(0.1, 2); fill(10, 3)]

In [ ]:
using ReachabilityAnalysis: add_dimension

function building_BLDC01()
    @load building_path A B
    n = size(A, 1)
    Ae = add_dimension(A)

    #initial states
    center_X0 = [fill(0.000225, 10); fill(0.0, 38)]
    radius_X0 = [fill(0.000025, 10); fill(0.0, 14); 0.0001; fill(0.0, 23)]
    X0 = Hyperrectangle(center_X0, radius_X0)
    U = Hyperrectangle(low=[0.8], high=[1.0])
    X0 = X0 * U

    prob_BLDC01 = @ivp(x' = Ae * x, x ∈ X0)
end

In [15]:
@load "building_new.jld2" A B
n = size(A, 1)
U = Interval(0.8, 1.0)

## initial states
center_X0 = [fill(0.000225, 10); fill(0.0, 38)]
radius_X0 = [fill(0.000025, 10); fill(0.0, 14); 0.0001; fill(0.0, 23)]
X0 = Hyperrectangle(center_X0, radius_X0)
#prob = @ivp(x' = Ax + Bu, x(0) ∈ X0, u ∈ U, x ∈ Universe(n))

#prob_BLDC01 = homogeneize(prob)

LoadError: MethodError: no method matching homogeneize(::InitialValueProblem{ConstrainedLinearControlContinuousSystem{Float64, Matrix{Float64}, Matrix{Float64}, Universe{Float64}, Interval{Float64, IntervalArithmetic.Interval{Float64}}}, Hyperrectangle{Float64, Vector{Float64}, Vector{Float64}}})
[0mClosest candidates are:
[0m  homogeneize([91m::InitialValueProblem{ConstrainedLinearControlContinuousSystem{N, MT, IdentityMultiple{N}, XT, ConstantInput{SI}}, ST}[39m) where {N, MT<:AbstractMatrix{N}, XT<:LazySet{N}, SI<:(Singleton{N, VN} where VN<:AbstractVector{N}), ST<:LazySet{N}} at /home/mforets/.julia/dev/ReachabilityAnalysis/src/Continuous/normalization.jl:588

In [ ]:
using Plots

# ### BLDF01

prob_BLDF01 = building_BLDF01()

# #### Dense time

In [ ]:
@time sol_BLDF01_dense = solve(prob_BLDF01, T=20.0,
                               alg=LGG09(δ=0.004, template=[x25, -x25]));

In [ ]:
using LazySets.Arrays

n = statedim(prob_BLDF01)
const dire_sev = [SingleEntryVector(25, n, 1.0), SingleEntryVector(25, n, -1.0)]

@time sol_BLDF01_dense = solve(prob_BLDF01, T=20.0,
                               alg=LGG09(δ=0.004, template=dire_sev));

In [ ]:
alg = LGG09(δ=0.004, vars=(25), n=48)
@time sol_BLDF01_dense = solve(prob_BLDF01, T=20.0, alg=alg);

In [ ]:
n = dim(sol_BLDF01_dense)
@time sol_BLDF01_dense = solve(prob_BLDF01, T=20.0, alg=LGG09(δ=0.004, vars=[25], n=n));

In [ ]:
flowpipe(sol_BLDF01_dense).ext[:alg_vars] # if this is not missing => use it !

## Use cases

- plot `sol_BLDF01_dense[1]`: TemplateReachSet

- plot `sol_BLDF01_dense[1:10]`: array of TemplateReachSet

- plot `flowpipe(sol_BLDF01_dense)`: flowpipe of TemplateReachSet

- plot `sol_BLDF01_dense`: ReachSolution of TemplateReachSet

Also related:

- concrete projection of a `TemplateReachSet`
- concrete projection of a vector of `TemplateReachSet`
- concrete projection of a flowpipe `TemplateReachSet`
- concrete projection of a solution `TemplateReachSet`

## Strategy

- only in the two cases we have access to the `alg_vars` data
- for the first two, we can quickly compare the plotting vars with the computed alg vars if the template uses SEV; otherwise it depends on the number of directions to check... so maybe it's not worth it

In [ ]:
R = sol_BLDF01_dense[1]

In [ ]:
project(R, (25))

In [ ]:
@which project(R, (25))

In [ ]:
@which project(set(R), [25])

In [ ]:
using BenchmarkTools

In [ ]:
@btime solve($prob_BLDF01, T=20.0, alg=LGG09(δ=0.004, template=[$x25, -$x25]));

In [ ]:
# Safety properties

@show ρ(x25, sol_BLDF01_dense)
#-
@show ρ(x25, sol_BLDF01_dense) <= 5.1e-3 # BLDF01 - BDS01
#-
@show ρ(x25, sol_BLDF01_dense) <= 4e-3 # BLDF01 - BDU01
#-
@show ρ(x25, sol_BLDF01_dense(20.0))
#-
@show ρ(x25, sol_BLDF01_dense(20.0)) <= -0.78e-3 # BLDF01 - BDU02

In [ ]:
sol_BLDF01_dense[1]

In [ ]:
plot(sol_BLDF01_dense, vars=(0, 25), lc=:blue, color=:blue, alpha=0.8, lw=1.0, xlab="t", ylab="x25")

In [ ]:

# #### Discrete time

sol_BLDF01_discrete = solve(prob_BLDF01, T=20.0, alg=LGG09(δ=0.01, template=[x25, -x25], approx_model=NoBloating()));

plot(sol_BLDF01_discrete, vars=(0, 25), linecolor=:blue, color=:blue, alpha=0.8, lw=1.0, xlab="t", ylab="x25")

# Safety properties

ρ(x25, sol_BLDF01_discrete)
#-
ρ(x25, sol_BLDF01_discrete) <= 5.1e-3 # BLDF01 - BDS01
#-
ρ(x25, sol_BLDF01_discrete)
#-
ρ(x25, sol_BLDF01_discrete) <= 4e-3 # BLDF01 - BDU01
#-
ρ(x25, sol_BLDF01_discrete(20.0))
#-
ρ(x25, sol_BLDF01_discrete(20.0)) <= -0.78e-3 # BLDF01 - BDU02

# ### BLDC01

prob_BLDC01 = building_BLDC01()

# #### Dense time

sol_BLDC01_dense = solve(prob_BLDC01, T=20.0, alg=LGG09(δ=0.006, template=[x25e, -x25e]))

plot(sol_BLDC01_dense, vars=(0, 25), linecolor=:blue, color=:blue, alpha=0.8, lw=1.0, xlab="t", ylab="x25")

# Safety properties

ρ(x25e, sol_BLDC01_dense)
#-
ρ(x25e, sol_BLDC01_dense) <= 5.1e-3 # BLDC01 - BDS01
#-
ρ(x25e, sol_BLDC01_dense) <= 4e-3 # BLDC01 - BDU01
#-
ρ(x25, sol_BLDF01_discrete(20.0))
#-
ρ(x25e, sol_BLDC01_dense(20.0)) <= -0.78e-3 # BLDC01 - BDU02

# #### Discrete time

sol_BLDC01_discrete = solve(prob_BLDC01, T=20.0, alg=LGG09(δ=0.01, template=[x25e, -x25e], approx_model=NoBloating()))

plot(sol_BLDC01_discrete, vars=(0, 25), linecolor=:blue, color=:blue, alpha=0.8, lw=1.0, xlab="t", ylab="x25")

# Safety properties

ρ(x25e, sol_BLDC01_discrete)
#-
ρ(x25e, sol_BLDC01_discrete) <= 5.1e-3 # BLDC01 - BDS01
#-
ρ(x25e, sol_BLDC01_discrete)
#-
ρ(x25e, sol_BLDC01_discrete) <= 4e-3 # BLDC01 - BDU01
#-
ρ(x25e, sol_BLDC01_discrete(20.0))
#-
ρ(x25e, sol_BLDC01_discrete(20.0)) <= -0.78e-3 # BLDC01 - BDU02

# ## References

# [^]:
